<a href="https://colab.research.google.com/github/SUMANASHRI/AML-Suspicious-Detection/blob/main/AML_Suspicious_Detection_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas pandasql numpy

import pandas as pd
import numpy as np
import pandasql as ps
from datetime import datetime, timedelta


  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26773 sha256=44eb0ffc0337b23e569b6d18ce1886dfd9e6d4b5ced107ce830864aa48efe423
  Stored in directory: /root/.cache/pip/wheels/15/a1/e7/6f92f295b5272ae5c02365e6b8fa19cb93f16a537090a1cf27
Successfully built pandasql


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/aml_dataset_latest.csv')

In [6]:
df.head()

,customer_id,transaction_amount,transaction_type,account_balance,timestamp
0,1048,193036,UPI,200657,2024-07-11 05:22:48.348295
1,1048,156860,UPI,223375,2024-06-22 18:29:21.352230
2,1019,14050,CASH,22781,2024-04-13 00:21:59.533833
3,1040,170455,CASH,465082,2024-02-15 17:12:33.907670
4,1039,192884,NEFT,398829,2024-09-20 12:11:50.097098


In [9]:
df.shape

(500, 5)

In [10]:
# Remove duplicates
df = df.drop_duplicates()

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort by timestamp
df = df.sort_values(by="timestamp")

df.reset_index(drop=True, inplace=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   customer_id         500 non-null    int64         
 1   transaction_amount  500 non-null    int64         
 2   transaction_type    500 non-null    object        
 3   account_balance     500 non-null    int64         
 4   timestamp           500 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 19.7+ KB


In [11]:
# Average transaction amount per customer
df["avg_amount"] = df.groupby("customer_id")["transaction_amount"].transform("mean")

# Time difference between customer transactions
df["time_diff"] = df.groupby("customer_id")["timestamp"].diff()

# Replace missing differences
df["time_diff"].fillna(pd.Timedelta(seconds=999999), inplace=True)


/tmp/ipython-input-3723316534.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["time_diff"].fillna(pd.Timedelta(seconds=999999), inplace=True)


In [12]:
df["rule_high_value"] = df["transaction_amount"] > 100000


In [13]:
df["rule_rapid"] = df["time_diff"].dt.seconds < 60



In [14]:
df["rule_abrupt"] = df["transaction_amount"] > (3 * df["avg_amount"])


In [15]:
df["rule_cash"] = (df["transaction_type"] == "CASH") & (df["transaction_amount"] > 50000)


In [16]:
df["suspicious"] = df[[
    "rule_high_value",
    "rule_rapid",
    "rule_abrupt",
    "rule_cash"
]].any(axis=1)

df.head()


,customer_id,transaction_amount,transaction_type,account_balance,timestamp,avg_amount,time_diff,rule_high_value,rule_rapid,rule_abrupt,rule_cash,suspicious
0,1044,106860,UPI,457373,2024-01-03 09:14:05.002285,75347.000000,11 days 13:46:39,True,False,False,False,True
1,1007,1061,CASH,173041,2024-01-03 12:38:43.041910,81629.500000,11 days 13:46:39,False,False,False,False,False
2,1016,69888,CASH,294519,2024-01-04 00:26:59.608919,92469.100000,11 days 13:46:39,False,False,False,True,True
3,1022,108928,UPI,66850,2024-01-05 10:26:26.780198,108549.307692,11 days 13:46:39,True,False,False,False,True
4,1033,182837,IMPS,392704,2024-01-05 11:48:15.150019,104218.363636,11 days 13:46:39,True,False,False,False,True


In [17]:
query = """
SELECT customer_id, transaction_amount, transaction_type, timestamp,
       rule_high_value, rule_rapid, rule_abrupt, rule_cash
FROM df
WHERE suspicious = 1
ORDER BY customer_id, timestamp;
"""

suspicious_df = ps.sqldf(query)
suspicious_df.head(20)


/tmp/ipython-input-3448370596.py:9: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  suspicious_df = ps.sqldf(query)


,customer_id,transaction_amount,transaction_type,timestamp,rule_high_value,rule_rapid,rule_abrupt,rule_cash
0,1001,167716,CASH,2024-02-01 16:10:05.320632,1,0,0,1
1,1001,173104,NEFT,2024-05-13 01:09:28.161841,1,0,0,0
2,1001,160211,UPI,2024-05-19 11:35:32.481463,1,0,0,0
3,1001,123211,CASH,2024-06-25 17:10:17.295160,1,0,0,1
4,1001,134790,IMPS,2024-08-08 22:24:40.736481,1,0,0,0
5,1001,148975,UPI,2024-08-13 18:24:27.766922,1,0,0,0
6,1001,98661,CASH,2024-08-24 00:11:21.429766,0,0,0,1
7,1002,175503,UPI,2024-02-10 09:11:47.040445,1,0,0,0
8,1002,113901,NEFT,2024-02-29 04:02:44.136720,1,0,0,0
9,1002,164450,UPI,2024-03-12 13:42:52.663113,1,0,0,0


In [18]:
suspicious_df.to_csv("suspicious_report.csv", index=False)
print("Report Generated!")


Report Generated!


In [20]:
from google.colab import files

In [21]:
files.download("suspicious_report.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
total = len(df)
suspicious_count = df["suspicious"].sum()

print("Total Transactions:", total)
print("Suspicious Transactions:", suspicious_count)
print("Percentage Flagged:", round((suspicious_count/total)*100, 2), "%")


Total Transactions: 500
Suspicious Transactions: 285
Percentage Flagged: 57.0 %
